In [1]:
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
#import fastscape
import xarray as xr
%matplotlib inline

In [2]:
print('xarray-simlab version: ', xs.__version__)
print('xarray version: ', xr.__version__)
#print('fastscape version: ', fastscape.__version__)

xarray-simlab version:  0.3.0+24.gdfced96
xarray version:  0.14.1


In [3]:
from fastscape.models import marine_model

ModuleNotFoundError: No module named 'fastscape'

In [4]:
marine_model

NameError: name 'marine_model' is not defined

In [5]:
marine_model.visualize(show_inputs=True)

NameError: name 'marine_model' is not defined

In [8]:
in_ds = xs.create_setup(
    model=marine_model,
    clocks={
        'time': np.arange(0, 4e6 + 1e3, 1e3),
        'out': np.arange(0, 4e6 + 2e5, 2e5),
    },
    master_clock='time',
    input_vars={
        'grid__shape': ('shape_yx', [101, 101]),
        'grid__length': ('shape_yx', [4e5, 4e5]),
        'boundary__status': ('border', ['fixed_value', 'whatever', 'looped', 'looped']),
        'init_topography': {
            'x_left': 0,
            'x_right': 2.5e5,
            'elevation_left': -1e3,
            'elevation_right': 0.
        },
        'uplift': {
            'x_position': 2.5e5,
            'rate_left': 0.,
            'rate_right': 3e-4
        },
        'flow__slope_exp': 1.,
        'spl': {
            'k_coef_bedrock': 1e-5,
            'k_coef_soil': 1e-5,
            'g_coef_bedrock': 1.,
            'g_coef_soil': 1.,
            'area_exp': 0.4,
            'slope_exp': 1.
        },
        'diffusion': {
            'diffusivity_bedrock': 1e-2,
            'diffusivity_soil': 1e-2
        },
        'sea__level': 0.,
        'marine': {
            'ss_ratio_land': 0.5,
            'porosity_sand': 0.,
            'porosity_silt': 0.,
            'e_depth_sand': 1e3,
            'e_depth_silt': 1e3,
            'diffusivity_sand': 5e2,
            'diffusivity_silt': 2.5e2,
            'layer_depth': 1e2,
        },
        'strati': {
            'freeze_time': ('horizon', np.linspace(0, 4e6, 5))
        }
    },
    output_vars={
        'out': ['topography__elevation',
                'strati__elevation',
                'marine__ss_ratio_sea'],
        None: ['boundary__border',
               'grid__x',
               'grid__y'],
    }
)

in_ds

<xarray.Dataset>
Dimensions:                           (border: 4, horizon: 5, out: 21, shape_yx: 2, time: 4001)
Coordinates:
  * time                              (time) float64 0.0 1e+03 ... 4e+06
  * out                               (out) float64 0.0 2e+05 ... 3.8e+06 4e+06
Dimensions without coordinates: border, horizon, shape_yx
Data variables:
    grid__shape                       (shape_yx) int64 101 101
    grid__length                      (shape_yx) float64 4e+05 4e+05
    boundary__status                  (border) <U11 'fixed_value' ... 'looped'
    init_topography__x_left           int64 0
    init_topography__x_right          float64 2.5e+05
    init_topography__elevation_left   float64 -1e+03
    init_topography__elevation_right  float64 0.0
    uplift__x_position                float64 2.5e+05
    uplift__rate_left                 float64 0.0
    uplift__rate_right                float64 0.0003
    flow__slope_exp                   float64 1.0
    spl__k_coef_bedrock   

In [9]:
out_ds = (in_ds.xsimlab.run(model=marine_model)
               .set_index(x='grid__x', y='grid__y',
                          border='boundary__border',
                          horizon='strati__freeze_time'))

In [10]:
out_ds

<xarray.Dataset>
Dimensions:                           (border: 4, horizon: 5, out: 21, shape_yx: 2, time: 4001, x: 101, y: 101)
Coordinates:
  * time                              (time) float64 0.0 1e+03 ... 4e+06
  * out                               (out) float64 0.0 2e+05 ... 3.8e+06 4e+06
  * x                                 (x) float64 0.0 4e+03 ... 3.96e+05 4e+05
  * y                                 (y) float64 0.0 4e+03 ... 3.96e+05 4e+05
  * border                            (border) object 'left' ... 'bottom'
  * horizon                           (horizon) float64 0.0 1e+06 ... 4e+06
Dimensions without coordinates: shape_yx
Data variables:
    grid__shape                       (shape_yx) int64 101 101
    grid__length                      (shape_yx) float64 4e+05 4e+05
    boundary__status                  (border) <U11 'fixed_value' ... 'looped'
    init_topography__x_left           int64 0
    init_topography__x_right          float64 2.5e+05
    init_topography__elevatio

In [11]:
import hvplot.xarray
import holoviews as hv
from xshade import hillshade


ss_ratio_plot = out_ds.marine__ss_ratio_sea.hvplot.image(
    x='x', y='y', clim=(0, 1),
    width=550, height=450,
    cmap=plt.cm.viridis_r, groupby='out'
)

hillshade_plot = hillshade(out_ds, 'out').hvplot.image(
    x='x', y='y', cmap=plt.cm.gray, alpha=0.5,
    colorbar=False, hover=False, groupby='out'
)

yhorizons_plot = out_ds.strati__elevation.sel(y=2e5).hvplot.line(
    by='horizon', groupby='out', ylim=(-1000, 1000),
    width=550, legend='bottom_right',
)

hv.Layout((ss_ratio_plot * hillshade_plot) + yhorizons_plot).cols(1)

ModuleNotFoundError: No module named 'hvplot'